In [7]:
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

In [8]:
AZURE_DIRECTORY = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aimethica-general/code/Users/alexandru.popescu/anomaly_DOCUMENTE_ORDINE_DE_INCASARE'

In [9]:
subscription_id = "9107822e-69ab-454f-a905-b59c4e15c1af"
resource_group = "AiMethica-ML"
workspace = "AiMethica-AZML"

In [10]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [11]:
ws = ml_client.workspaces.get(workspace)
print(ws.location, ":", ws.resource_group)

westeurope : AiMethica-ML


In [12]:
custom_env_name = 'aimethica-anomaly-env'
job_name = 'gifted_boat_1j947xhlqf'
env_name = custom_env_name + "@latest"

In [13]:
from azure.ai.ml.entities import Model

model = Model(
    # the script stores the model as "model"
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/outputs/",
    name="anomaly_document_facturi_export",
    description="Anomaly detection model for the DOCUMENT ORDINE DE INCASARE dataset.",
    type="custom_model"
)

In [14]:
registered_model = ml_client.models.create_or_update(model=model)

In [15]:
online_endpoint_name = "anomaly-document-ordine-incasare"

In [16]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Anomaly detection model for the DOCUMENT ORDINE DE INCASARE dataset.",
    auth_mode="key"
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpint anomaly-document-ordine-incasare provisioning state: Succeeded


In [17]:
model_name = registered_model.name + '@latest'

code_directory = os.path.join(AZURE_DIRECTORY,'src')

blue_deployment = ManagedOnlineDeployment(
    name="tff-blue",
    endpoint_name=online_endpoint_name,
    model=model_name,
    code_configuration=CodeConfiguration(code=code_directory, scoring_script="score.py"),
    environment=env_name,
    instance_type="Standard_DS1_v2",
    instance_count=1,
)

In [18]:
blue_deployment = ml_client.begin_create_or_update(blue_deployment)


Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint anomaly-document-ordine-incasare exists
Uploading src (0.01 MBs): 100%|██████████| 6019/6019 [00:00<00:00, 163663.05it/s]




.

In [19]:
anomaly_request = os.path.join(AZURE_DIRECTORY,"data/anomaly_request.json")
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="tff-blue",
    request_file=anomaly_request
)

'{"response": [false, false, false, false]}'